# Uso de los modelos baseline y modelo usando EfficientNet-B0

In [ ]:
!pip install -q timm torch torchvision pillow matplotlib

## Importar librerias

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import timm

## Definir arquitecturas

### Baseline

In [ ]:
class BaselineCNN(nn.Module):

    def __init__(self, num_classes):
        super(BaselineCNN, self).__init__()

        # Bloques convulocionales
        self.features = nn.Sequential(
            # Bloque 1: 224 -> 112
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.RelU(inplace=True),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Bloque 2: 112 -> 56
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.RelU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Bloque 3: 56 -> 28
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.RelU(inplace=True),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Bloque 4: 28 -> 14
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.RelU(inplace=True),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Bloque 5: 14 -> 7
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
        )

        # pooling y classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


### Transfer Learning

In [ ]:
class MushroomClassifier(nn.Module):
  def __init__(self, num_classes, pretrained=True):
    super(MushroomClassifier, self).__init__()

    # Cargar EfficientNet-B0
    self.backbone = timm.create_model(
        'efficientnet_b0',
        pretrained=pretrained,
        num_classes=num_classes
    )

  def forward(self,x):
    return self.backbone(x)

In [ ]:
from google.colab import files
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# uploaded_csv = files.upload()

# Get the uploaded CSV filename
csv_filename = 'train.csv'

# Load CSV and create label encoder
train_df = pd.read_csv(csv_filename)
species_list = sorted(train_df.iloc[:, 1].unique())

# Create label encoder
label_encoder = LabelEncoder()
label_encoder.fit(species_list)

num_classes = len(species_list)

## Subir pesos

In [ ]:
from google.colab import files

uploaded = files.upload()

baseline_path = 'baseline_best.pth'
transfer_path = 'efficientNet_best.pth'

Saving baseline_best.pth to baseline_best.pth
Saving efficientNet_best.pth to efficientNet_best.pth


## Cargar modelos

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\nUsando dispositivo: {device}\n")

print("="*70)
print("CARGANDO MODELOS")
print("="*70)

# Cargar modelo de transfer learning
print("\n1. Cargando modelo de Transfer Learning...")
transfer_checkpoint = torch.load(transfer_path, map_location=device, weights_only=False)

transfer_model = MushroomClassifier(num_classes=num_classes, pretrained=False)
transfer_model.load_state_dict(transfer_checkpoint['model_state_dict'])
transfer_model = transfer_model.to(device)
transfer_model.eval()

print(f"   ✓ Modelo de Transfer Learning cargado")
if 'test_acc' in transfer_checkpoint:
    print(f"   • Test Accuracy: {transfer_checkpoint['test_acc']:.2f}%")
elif 'val_acc' in transfer_checkpoint:
    print(f"   • Val Accuracy: {transfer_checkpoint['val_acc']:.2f}%")

# Cargar modelo baseline
print("\n2. Cargando modelo Baseline...")
baseline_checkpoint = torch.load(baseline_path, map_location=device, weights_only=False)

baseline_model = BaselineCNN(num_classes=num_classes)
baseline_model.load_state_dict(baseline_checkpoint['model_state_dict'])
baseline_model = baseline_model.to(device)
baseline_model.eval()

print(f"   ✓ Modelo Baseline cargado")
if 'val_acc' in baseline_checkpoint:
    print(f"   • Val Accuracy: {baseline_checkpoint['val_acc']:.2f}%")

print("\n" + "="*70)
print(f"¡LISTO PARA CLASIFICAR {num_classes} ESPECIES DE HONGOS!")
print("="*70 + "\n")


Usando dispositivo: cuda

CARGANDO MODELOS

1. Cargando modelo de Transfer Learning...
   ✓ Modelo de Transfer Learning cargado
   • Val Accuracy: 72.79%

2. Cargando modelo Baseline...
   ✓ Modelo Baseline cargado
   • Val Accuracy: 19.64%

¡LISTO PARA CLASIFICAR 169 ESPECIES DE HONGOS!



## Preprocesamiento

In [ ]:
img_size = 224
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
])

## Funciones para predicir con ambos modelos

In [ ]:
def predict_both_models(image_path, label_encoder_param, top_k=5):
    """
    Predecir usando AMBOS modelos y comparar resultados

    Args:
        image_path: ruta a imagen o PIL Image
        label_encoder_param: label encoder para decodificar predicciones
        top_k: número de predicciones a mostrar

    Returns:
        image, baseline_predictions, transfer_predictions
    """
    # Cargar imagen
    if isinstance(image_path, str):
        image = Image.open(image_path).convert('RGB')
    else:
        image = image_path.convert('RGB')

    # Preprocesar
    image_tensor = transform(image).unsqueeze(0).to(device)

    # Predecir con AMBOS modelos
    with torch.no_grad():
        # Predicciones Baseline
        baseline_outputs = baseline_model(image_tensor)
        baseline_probs = torch.softmax(baseline_outputs, dim=1)
        baseline_top_probs, baseline_top_indices = baseline_probs.topk(top_k, dim=1)

        # Predicciones Transfer Learning
        transfer_outputs = transfer_model(image_tensor)
        transfer_probs = torch.softmax(transfer_outputs, dim=1)
        transfer_top_probs, transfer_top_indices = transfer_probs.topk(top_k, dim=1)

    # Convertir a numpy
    baseline_top_probs = baseline_top_probs.cpu().numpy()[0]
    baseline_top_indices = baseline_top_indices.cpu().numpy()[0]
    transfer_top_probs = transfer_top_probs.cpu().numpy()[0]
    transfer_top_indices = transfer_top_indices.cpu().numpy()[0]

    # Decodificar etiquetas
    baseline_predictions = []
    for prob, idx in zip(baseline_top_probs, baseline_top_indices):
        species = label_encoder_param.inverse_transform([idx])[0]
        baseline_predictions.append((species, prob * 100))

    transfer_predictions = []
    for prob, idx in zip(transfer_top_probs, transfer_top_indices):
        species = label_encoder_param.inverse_transform([idx])[0]
        transfer_predictions.append((species, prob * 100))

    return image, baseline_predictions, transfer_predictions

In [ ]:
def visualize_comparison(image, baseline_preds, transfer_preds):
    """
    Visualizar imagen y predicciones de ambos modelos lado a lado
    """
    fig = plt.figure(figsize=(20, 7))
    gs = fig.add_gridspec(1, 3, width_ratios=[1, 1, 1], wspace=0.3)

    # Mostrar imagen
    ax_img = fig.add_subplot(gs[0])
    ax_img.imshow(image)
    ax_img.set_title('Imagen de Hongo', fontsize=14, fontweight='bold', pad=15)
    ax_img.axis('off')

    # Predicciones Baseline
    ax_baseline = fig.add_subplot(gs[1])
    species_names = [p[0][:35] + '...' if len(p[0]) > 35 else p[0] for p in baseline_preds]
    confidences = [p[1] for p in baseline_preds]

    y_pos = np.arange(len(species_names))
    bars = ax_baseline.barh(y_pos, confidences, color='#e74c3c', alpha=0.7)
    ax_baseline.set_yticks(y_pos)
    ax_baseline.set_yticklabels(species_names, fontsize=9)
    ax_baseline.set_xlabel('Confianza (%)', fontsize=11, fontweight='bold')
    ax_baseline.set_title('Baseline CNN\n(Entrenado desde cero)',
                          fontsize=13, fontweight='bold', pad=15)
    ax_baseline.set_xlim(0, 100)
    ax_baseline.invert_yaxis()
    ax_baseline.grid(axis='x', alpha=0.3, linestyle='--')

    # Agregar etiquetas de porcentaje
    for i, (bar, conf) in enumerate(zip(bars, confidences)):
        ax_baseline.text(conf + 2, i, f'{conf:.1f}%',
                        va='center', fontsize=8, fontweight='bold')

    # Predicciones Transfer Learning
    ax_transfer = fig.add_subplot(gs[2])
    species_names = [p[0][:35] + '...' if len(p[0]) > 35 else p[0] for p in transfer_preds]
    confidences = [p[1] for p in transfer_preds]

    bars = ax_transfer.barh(y_pos, confidences, color='#2ecc71', alpha=0.7)
    ax_transfer.set_yticks(y_pos)
    ax_transfer.set_yticklabels(species_names, fontsize=9)
    ax_transfer.set_xlabel('Confianza (%)', fontsize=11, fontweight='bold')
    ax_transfer.set_title('Transfer Learning\n(EfficientNet-B0 + ImageNet)',
                         fontsize=13, fontweight='bold', pad=15)
    ax_transfer.set_xlim(0, 100)
    ax_transfer.invert_yaxis()
    ax_transfer.grid(axis='x', alpha=0.3, linestyle='--')

    # Agregar etiquetas de porcentaje
    for i, (bar, conf) in enumerate(zip(bars, confidences)):
        ax_transfer.text(conf + 2, i, f'{conf:.1f}%',
                        va='center', fontsize=8, fontweight='bold')

    plt.tight_layout()
    plt.show()

    # Imprimir comparación
    print("\n" + "="*90)
    print("COMPARACIÓN DE PREDICCIONES")
    print("="*90)
    print(f"{'Rank':<6} {'Baseline CNN':<45} {'Transfer Learning':<45}")
    print("-"*90)

    max_len = max(len(baseline_preds), len(transfer_preds))
    for i in range(max_len):
        rank = f"#{i+1}"

        if i < len(baseline_preds):
            b_species, b_conf = baseline_preds[i]
            b_text = f"{b_species[:35]:35s} ({b_conf:5.2f}%)"
        else:
            b_text = "-"

        if i < len(transfer_preds):
            t_species, t_conf = transfer_preds[i]
            t_text = f"{t_species[:35]:35s} ({t_conf:5.2f}%)"
        else:
            t_text = "-"

        print(f"{rank:<6} {b_text:<45} {t_text:<45}")

    print("="*90)

    # Destacar si las predicciones coinciden
    if baseline_preds[0][0] == transfer_preds[0][0]:
        print(f"\n✓ AMBOS MODELOS CONCUERDAN: {baseline_preds[0][0]}")
    else:
        print(f"\n⚠️  LOS MODELOS NO CONCUERDAN:")
        print(f"   Baseline predice:  {baseline_preds[0][0]} ({baseline_preds[0][1]:.1f}%)")
        print(f"   Transfer predice:  {transfer_preds[0][0]} ({transfer_preds[0][1]:.1f}%)")

    # Mostrar diferencia de confianza
    print(f"\nComparación de confianza:")
    print(f"   Predicción top Baseline: {baseline_preds[0][1]:.1f}%")
    print(f"   Predicción top Transfer: {transfer_preds[0][1]:.1f}%")
    print(f"   Diferencia: {abs(transfer_preds[0][1] - baseline_preds[0][1]):.1f}%")
    print("="*90 + "\n")




## Subir y compara una sola imagen

In [ ]:
print("Sube una imagen de hongo para clasificar:")
uploaded_images = files.upload()

# Obtener primer archivo subido
image_path = list(uploaded_images.keys())[0]
print(f"\n✓ Imagen subida: {image_path}")

# Hacer predicción con AMBOS modelos (PASS label_encoder)
print("\nAnalizando con ambos modelos...")
image, baseline_preds, transfer_preds = predict_both_models(
    image_path,
    label_encoder,  # ← PASS IT HERE
    top_k=5
)

# Visualizar comparación
visualize_comparison(image, baseline_preds, transfer_preds)